In [28]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import random
import os
import natsort
import warnings
warnings.filterwarnings('ignore')
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import imutils
import dlib
import cv2 

predictor_model = "./landmarks.dat"

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor and the face aligner
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_model)
fa = FaceAligner(predictor, desiredFaceWidth=256)

def face_align(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 2)

    for rect in rects:
        (x, y, w, h) = rect_to_bb(rect)
        faceOrig = imutils.resize(image[y:y + h, x:x+w], width=256)
        faceAligned = fa.align(image, gray, rect)
        return faceAligned

##################################################################

# Define constants
FAST_RUN = False
IMAGE_WIDTH = 48
IMAGE_HEIGHT = 48
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=1
PIXELS = IMAGE_HEIGHT * IMAGE_WIDTH*3

# Create the model
model_emotion = Sequential()
model_emotion.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model_emotion.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_emotion.add(MaxPooling2D(pool_size=(2, 2)))
model_emotion.add(Dropout(0.25))
model_emotion.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_emotion.add(MaxPooling2D(pool_size=(2, 2)))
model_emotion.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_emotion.add(MaxPooling2D(pool_size=(2, 2)))
model_emotion.add(Dropout(0.25))
model_emotion.add(Flatten())
model_emotion.add(Dense(1024, activation='relu'))
model_emotion.add(Dropout(0.5))
model_emotion.add(Dense(7, activation='softmax'))
model_emotion.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_emotion.load_weights("emotion_model.h5")
#####################################################################

model_mask1 = Sequential()
model_mask1.add(Conv2D(32, kernel_size=(3, 3), activation = 'relu', input_shape = (128, 128, 3)))
model_mask1.add(MaxPooling2D(pool_size=(2, 2)))
model_mask1.add(Dropout(0.5))
model_mask1.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
model_mask1.add(MaxPooling2D(pool_size=(2, 2)))
model_mask1.add(Dropout(0.5))
model_mask1.add(Conv2D(128,kernel_size=(3, 3), activation='relu'))
model_mask1.add(MaxPooling2D(pool_size=(2, 2)))
model_mask1.add(Dropout(0.5))
model_mask1.add(Flatten())
model_mask1.add(Dense(512, activation='relu'))
model_mask1.add(Dropout(0.5))
model_mask1.add(Dense(2, activation = 'sigmoid'))
model_mask1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model_mask1.load_weights("mask_model_5.h5")

#####################################################################
model_glasses1 = Sequential()

model_glasses1.add(Conv2D(32, kernel_size=(3, 3), strides = (1,1), padding = 'same', activation = 'relu', input_shape = (128, 128, 3)))
model_glasses1.add(BatchNormalization())
model_glasses1.add(MaxPooling2D(pool_size=(2, 2)))
model_glasses1.add(Dropout(0.25))

model_glasses1.add(Conv2D(64,kernel_size=(3, 3), strides = (1,1),  padding = 'same', activation='relu'))
model_glasses1.add(BatchNormalization())
model_glasses1.add(MaxPooling2D(pool_size=(2, 2)))
model_glasses1.add(Dropout(0.25))

model_glasses1.add(Conv2D(128,kernel_size=(3, 3), strides = (1,1),  padding = 'same', activation='relu'))
model_glasses1.add(BatchNormalization())
model_glasses1.add(MaxPooling2D(pool_size=(2, 2)))
model_glasses1.add(Dropout(0.25))

model_glasses1.add(Flatten())
model_glasses1.add(Dense(512, activation='relu'))
model_glasses1.add(BatchNormalization())
model_glasses1.add(Dropout(0.5))
model_glasses1.add(Dense(2, activation = 'softmax'))

model_glasses1.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model_glasses1.load_weights("glasses_model_1.h5")

#####################################################################
model_glasses2 = Sequential()
model_glasses2.add(Conv2D(32, kernel_size=(3, 3), activation = 'relu', input_shape = (128, 128, 3)))
model_glasses2.add(MaxPooling2D(pool_size=(2, 2)))
model_glasses2.add(Dropout(0.5))
model_glasses2.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
model_glasses2.add(MaxPooling2D(pool_size=(2, 2)))
model_glasses2.add(Dropout(0.5))
model_glasses2.add(Conv2D(128,kernel_size=(3, 3), activation='relu'))
model_glasses2.add(MaxPooling2D(pool_size=(2, 2)))
model_glasses2.add(Dropout(0.5))
model_glasses2.add(Flatten())
model_glasses2.add(Dense(512, activation='relu'))
model_glasses2.add(Dropout(0.5))
model_glasses2.add(Dense(2, activation = 'sigmoid'))
model_glasses2.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model_glasses2.load_weights("glasses_model_2.h5")
#####################################################################
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 
                3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}


from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (128, 128, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image


file_path = 'C:/Users/face/'
file_path = '../graduation_dataset/mask/test/'
file_path = '../graduation_dataset/glasses/test_3/'
file_path = 'C:/Users/example/'
file_path = '../face-4263.png'


image = load(file_path)
result_skin = model
result_mask1 = model_mask1.predict(image)
result_glasses1 = model_glasses1.predict(image)
print(file_path)
if result_mask1[0][0] > result_mask1[0][1]:
    print('해당 사진은 마스크를 썼습니다.')
else:
    image2 = cv2.imread(filepath)
    faceAligned = face_align(image2)
    np_image = Image.fromarray(cv2.cvtColor(faceAligned, cv2.COLOR_BGR2GRAY))
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (48, 48, 1))
    np_image = np.expand_dims(np_image, axis=0)
    result = model_emotion.predict(np_image)
    print('예측: ' + filename)
    maxindex = int(np.argmax(result))
    print(emotion_dict[maxindex])
        
if result_glasses1[0][0] < result_glasses1[0][1]:
    print('해당 사진은 안경을 쓰지않았습니다.')
else:
    #print('해당 사진은 안경을 썼습니다.')
    result_glasses2 = model_glasses2.predict(image)
    if max(result_glasses2[0]) == result_glasses2[0][0]:
         print('해당 사진은 안경을 썼습니다.')
    else:
         print('해당 사진은 선글라스를 썼습니다.')
        
print('#####################################')





../face-4263.png
예측: sunglasses_074.png
Happy
해당 사진은 선글라스를 썼습니다.
#####################################
